In [1]:
import numpy as np
import ctypes, time, os, tqdm, pickle, sharedmem
import matplotlib.pyplot as plt
import popeye.og as og
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus, resample_stimulus
from popeye import css
from scipy.io import loadmat
import nibabel as nib
from nilearn import plotting
import multiprocessing as mp
from ipywidgets import interact, widgets
from concurrent.futures import ThreadPoolExecutor

# Load helper functions
from dataloader import set_paths, load_stimuli, copy_files
%load_ext autoreload
%autoreload 2

In [2]:
# Initialize parameters
params = {}
params['subjID'] = 'JC'
# Got these from Zhengang, and he got it from rsvp_params.txt
params['viewingDistance'] = 83.5 # in cm
params['screenWidth'] = 36.2 # in cm
params['scaleFactor'] = 1
params['resampleFactor'] = 1
params['dtype'] = ctypes.c_int16

p = set_paths(params)

In [3]:
bar, stim_params = load_stimuli(p)
bar = bar[:, :, 0:201]
# Mirror y axis
bar = np.flip(bar, axis=0)
# Transpose x and y for bar
# bar = np.transpose(bar, (1, 0, 2))

# bar = resample_stimulus(bar, params['resampleFactor'])

copy_files(p, params)

# Extract number of TRs
method = 'ss5'

func_data = nib.load(p['pRF_' + method])
f_header = func_data.header
params['tr_length'] = f_header['pixdim'][4]
params['voxel_size'] = [f_header['pixdim'][i] for i in range(1, 4)]
params['nTRs'] = func_data.shape[-1]

# model to fit to
scan_data = func_data.get_fdata()
# scan_data = utils.percent_change(scan_data, ax=-1) # Convert to pct signal change

# Ground truth model-fit from mrVista
popeye_fit_path = os.path.join(p['pRF_data'], 'JC', 'mrVistaFit', 'RF_' + method + '-fFit.nii.gz')
mrVista_fit = nib.load(popeye_fit_path).get_fdata()

brainmask_data = nib.load(p['pRF_brainmask']).get_fdata() != 0
# Resample brainmask if first 2 dimensions are twice the third dimension
if brainmask_data.shape[0] == 2*brainmask_data.shape[2]:
    brainmask_data = brainmask_data[::2, ::2, :]

Subject folder already exists


In [4]:
# create stimulus object from popeye
stimulus = VisualStimulus(bar,
                          params['viewingDistance'],
                          params['screenWidth'],
                          params['scaleFactor'],
                          params['tr_length'],
                          params['dtype'],
)
                        # 'blinear')

In [ ]:
plt.figure()
plt.imshow(stimulus.deg_x, cmap='gray')
plt.show()

In [ ]:
# Crete an animation through time for stimulus.stim_arr whose third axis is time
def animate(time):
    plt.imshow(stimulus.stim_arr[:, :, time], cmap='gray')
    plt.show()

interact(animate, time=widgets.IntSlider(min=0, max=stimulus.stim_arr.shape[-1]-1, step=1, value=0))

In [5]:
# Testing only on visual ROIs
# Load visual ROIs
lh_v1 = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V1.nii.gz')).get_fdata()
lh_v2d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V2d.nii.gz')).get_fdata()
lh_v3d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V3d.nii.gz')).get_fdata()
lh_v3ab = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V3AB.nii.gz')).get_fdata()
rh_v1 = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V1.nii.gz')).get_fdata()
rh_v2d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V2d.nii.gz')).get_fdata()
rh_v3d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V3d.nii.gz')).get_fdata()
rh_v3ab = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V3AB.nii.gz')).get_fdata()
# Combine all ROIs using boolean OR
visual_rois = lh_v1 + lh_v2d + lh_v3d + lh_v3ab + rh_v1 + rh_v2d + rh_v3d + rh_v3ab
visual_rois = visual_rois > 0
visual_rois = lh_v1 + rh_v1
visual_rois = visual_rois > 0

In [ ]:
# # Run the fit
# start_time = time.time()
# ncpu = 1#int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
# print(f"Popeye will analyze {np.sum(visual_rois)} voxels using {ncpu} CPUs")
# pool = mp.Pool(ncpu)
# output = pool.map(utils.parallel_fit, bundle)
# # with mp.Pool(ncpu) as pool:
# #     results = list(tqdm.tqdm(pool.imap(utils.parallel_fit, bundle), total=len(bundle), desc='Fitting pRFs'))
# pool.close()
# pool.join()
# end_time = time.time()

In [ ]:
# # choose 10 voxels within the visual ROIs
# voxel_indices = np.argwhere(visual_rois)
# # np.random.shuffle(voxel_indices)
# voxel_indices = voxel_indices[:500]
# visual_rois = np.zeros_like(visual_rois)
# for idx in voxel_indices:
#     visual_rois[idx[0], idx[1],
#                 idx[2]] = 1

In [7]:
gaussian_model = og.GaussianModel(stimulus, utils.double_gamma_hrf, normalizer=utils.percent_change)
gaussian_model.hrf_delay = 0
gaussian_model.mask_size = 7

# Create scan data just for visual ROIs
scan_data_visual = scan_data.copy()
scan_data_visual[~visual_rois] = 0

[xi, yi, zi] = np.nonzero(visual_rois)
indices = [(xi[i], yi[i], zi[i]) for i in range(len(xi))]
num_voxels = len(indices)
timeseries_data = scan_data_visual[xi, yi, zi, :]

# Flush
func = []

# set search grids
x_grid = utils.grid_slice(-12.0, 12.0, 10)
y_grid = utils.grid_slice(-12.0, 12.0, 10)
s_grid = utils.grid_slice(1/gaussian_model.stimulus.ppd0, 3.0, 10)
# h_grid = utils.grid_slice(-1.0, 1.0, 3)
grids = (x_grid, y_grid, s_grid,)

# Set search bounds
x_bounds = (-15.0, 15.0)
y_bounds = (-15.0, 15.0)
s_bounds = (1/gaussian_model.stimulus.ppd0, 5.0)
# h_bounds = (0.001, 12.0)
b_bounds = (1e-8, None)
m_bounds = (None, None)
bounds = (x_bounds, y_bounds, s_bounds, b_bounds, m_bounds)

verbose = 0
auto_fit = 1
# bundle = utils.multiprocess_bundle(og.GaussianFit, gaussian_model, timeseries_data, grids, bounds, indices, auto_fit=autofit, verbose=verbose)

In [ ]:
# ncpus = sharedmem.cpu_count()
# print("popeye will analyze %d voxels across %d cores" %(len(bundle), ncpus))
# pool = mp.Pool(ncpus)
# t1 = time.time()
# output = pool.map(utils.parallel_fit, bundle)
# t2 = time.time()
# print(t2-t1)
# pool.close()
# pool.join()

In [ ]:
# # css_model = css.CompressiveSpatialSummationModel(stimulus, utils.spm_hrf)
# # css_model.hrf_delay = 0
# # Convolving the gaussian with hrf on each frame
# # css_model.mask_size = 5 # Maybe 3, 5 or 7
# gaussian_model = og.GaussianModel(stimulus, utils.double_gamma_hrf, normalizer=utils.percent_change)
# gaussian_model.hrf_delay = 0
# gaussian_model.mask_size = 6

# # Create scan data just for visual ROIs
# scan_data_visual = scan_data.copy()
# scan_data_visual[~visual_rois] = 0

# [xi, yi, zi] = np.nonzero(visual_rois)
# indices = [(xi[i], yi[i], zi[i]) for i in range(len(xi))]

# # set search grids
# # x_grid = (-5, 5)#(-12.4, 12.4) #(-5, 5)
# # y_grid = (-5, 5)#(-12.4, 12.4) #(-5, 5)
# # s_grid = (0.25, 2.0) #(0.25, 12.4)
# # n_grid = (0.05, 1.0)
# # grids = (x_grid, y_grid, s_grid, n_grid,)
# x_grid = utils.grid_slice(-5, 5, 3)
# y_grid = utils.grid_slice(-5, 5, 3)
# s_grid = utils.grid_slice(0.25, 5.25, 3)
# grids = (x_grid, y_grid, s_grid,)
# # set search bounds
# x_bounds = (-10, 10) #(-25, 25) #(-10, 10)
# y_bounds = (-10, 10)#(-25, 25) #(-10, 10)
# # sigma_bounds = (1/css_model.stimulus.ppd, 5.25)
# sigma_bounds = (0.001, 10)
# # n_bounds = (1e-8, 2.0)
# beta_bounds = (1e-8, None)
# baseline_bounds = (None, None)
# # bounds = (x_bounds, y_bounds, sigma_bounds, n_bounds, beta_bounds, baseline_bounds)
# bounds = (x_bounds, y_bounds, sigma_bounds, beta_bounds, baseline_bounds)

# # fit settings
# # Ns = 3
# auto_fit = 1
# verbose = 0

# # bundle = utils.multiprocess_bundle(css.CompressiveSpatialSummationFit, css_model, scan_data_visual, grids, bounds, indices, Ns, auto_fit, verbose)

In [ ]:
# Run the fit
# start_time = time.time()
# ncpu = 1#int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
# print(f"Popeye will analyze {np.sum(visual_rois)} voxels using {ncpu} CPUs")
# pool = mp.Pool(ncpu)
# output = pool.map(utils.parallel_fit, bundle)
# # with mp.Pool(ncpu) as pool:
# #     results = list(tqdm.tqdm(pool.imap(utils.parallel_fit, bundle), total=len(bundle), desc='Fitting pRFs'))
# pool.close()
# pool.join()
# end_time = time.time()

In [8]:
# Create a result holder
RF_ss5_fFit = np.empty((scan_data_visual.shape[0], scan_data_visual.shape[1], scan_data_visual.shape[2], 8))
vx_indices = np.argwhere(visual_rois)

In [9]:
def process_voxel(ix, iy, iz):
    if visual_rois[ix, iy, iz] == 1:
        th_vx_idx = np.where((vx_indices == (ix, iy, iz)).all(axis=1))[0][0]
        if np.mod(th_vx_idx, 100) == 0:
            run_time = time.time() - start_time
            if run_time < 60:
                print(f"Finished: {round(th_vx_idx/len(vx_indices)*100, 2)}%, time: {round(run_time, 2)} s")
            elif run_time < 3600:
                print(f"Finished: {round(th_vx_idx/len(vx_indices)*100, 2)}%, time: {round(run_time/60)} min {round(run_time%60)} s")
            else:
                print(f"Finished: {round(th_vx_idx/len(vx_indices)*100, 2)}%, time: {round(run_time/3600)} h {round(run_time%3600/60)} min {round(run_time%60)} s")
        voxel_data = scan_data[ix, iy, iz, :]
        fit = og.GaussianFit(
            gaussian_model,
            voxel_data,
            grids,
            bounds,
            (ix, iy, iz),
            auto_fit,
            verbose
        )
        return (ix, iy, iz, fit.theta, fit.rsquared, fit.rho, fit.sigma, fit.x, fit.y, fit.beta)
    return None

start_time = time.time()

futures = []
with ThreadPoolExecutor() as executor:
    for ix in range(visual_rois.shape[0]):
        for iy in range(visual_rois.shape[1]): 
            for iz in range(visual_rois.shape[2]):
                futures.append(executor.submit(process_voxel, ix, iy, iz))

results = [f.result() for f in futures]

for result in results:
    if result is not None:
        ix, iy, iz, theta, rsquared, rho, sigma, x, y, beta = result
        RF_ss5_fFit[ix, iy, iz, 0] = theta
        RF_ss5_fFit[ix, iy, iz, 1] = rsquared
        RF_ss5_fFit[ix, iy, iz, 2] = rho
        RF_ss5_fFit[ix, iy, iz, 3] = sigma
        RF_ss5_fFit[ix, iy, iz, 4] = 1
        RF_ss5_fFit[ix, iy, iz, 5] = x
        RF_ss5_fFit[ix, iy, iz, 6] = y
        RF_ss5_fFit[ix, iy, iz, 7] = beta

end_time = time.time()
print()
print(f"Time taken for fitting: {end_time - start_time}")

Finished: 0.0%, time: 8.88 s
Finished: 3.69%, time: 3 min 2 s
Finished: 7.39%, time: 6 min 43 s
Finished: 11.08%, time: 9 min 44 s
Finished: 14.78%, time: 11 min 30 s
Finished: 18.47%, time: 14 min 12 s
Finished: 22.16%, time: 17 min 10 s
Finished: 25.86%, time: 20 min 52 s


In [ ]:
# def process_voxel(ix, iy, iz):
#     if visual_rois[ix, iy, iz] == 1:
#         th_vx_idx = np.where((vx_indices == (ix, iy, iz)).all(axis=1))[0][0]
#         if np.mod(th_vx_idx, 100) == 0:
#             run_time = time.time() - start_time
#             if run_time < 60:
#                 print(f"Finished: {round(th_vx_idx/len(vx_indices)*100, 2)}%, time: {round(run_time, 2)} s")
#             elif run_time < 3600:
#                 print(f"Finished: {round(th_vx_idx/len(vx_indices)*100, 2)}%, time: {round(run_time/60)} min {round(run_time%60)} s")
#             else:
#                 print(f"Finished: {round(th_vx_idx/len(vx_indices)*100, 2)}%, time: {round(run_time/3600)} h {round(run_time%3600/60)} min {round(run_time%60)} s")
        
#         voxel_data = scan_data[ix, iy, iz, :]
#         fit = css.CompressiveSpatialSummationFit(
#             css_model,
#             voxel_data,
#             grids,
#             bounds,
#             (ix, iy, iz),
#             Ns,
#             auto_fit,
#             verbose
#         )
#         return (ix, iy, iz, fit.theta, fit.rsquared, fit.rho, fit.sigma, fit.n, fit.x, fit.y, fit.beta)
#     return None

# start_time = time.time()

# futures = []
# with ThreadPoolExecutor() as executor:
#     for ix in range(visual_rois.shape[0]):
#         for iy in range(visual_rois.shape[1]):
#             for iz in range(visual_rois.shape[2]):
#                 futures.append(executor.submit(process_voxel, ix, iy, iz))

# results = [f.result() for f in futures]

# for result in results:
#     if result is not None:
#         ix, iy, iz, theta, rsquared, rho, sigma, n, x, y, beta = result
#         RF_ss5_fFit[ix, iy, iz, 0] = theta
#         RF_ss5_fFit[ix, iy, iz, 1] = rsquared
#         RF_ss5_fFit[ix, iy, iz, 2] = rho
#         RF_ss5_fFit[ix, iy, iz, 3] = sigma
#         RF_ss5_fFit[ix, iy, iz, 4] = n
#         RF_ss5_fFit[ix, iy, iz, 5] = x
#         RF_ss5_fFit[ix, iy, iz, 6] = y
#         RF_ss5_fFit[ix, iy, iz, 7] = beta

# end_time = time.time()
# print(f"Time taken for fitting: {end_time - start_time}")

In [ ]:
# Save the results
# anat_data = nib.load(p['pRF_anat']) #.get_fdata()
# popeye_fit = nib.Nifti1Image(RF_ss5_fFit, affine=anat_data.affine, header=anat_data.header)
popeye_fit = nib.nifti1.Nifti1Image(RF_ss5_fFit, affine=func_data.affine, header=func_data.header)
if not os.path.exists(os.path.join(p['pRF_data'], params['subjID'], 'popeyeFit')):
    os.makedirs(os.path.join(p['pRF_data'], params['subjID'], 'popeyeFit'))
nib.save(popeye_fit, os.path.join(p['pRF_data'], params['subjID'], 'popeyeFit', 'RF_ss5_fFit_popeye.nii.gz'))

In [ ]:
# Assume visual_rois, mrVista_fit, and RF_ss5_fFit are already defined
nrows = 4
ncols = 2
fig, axs = plt.subplots(nrows, ncols, figsize=(10, 20))

titles = ['Theta', 'R-squared', 'Rho', 'Sigma', 'n', 'x', 'y', 'beta']
limits = [
    (0, 6),  # Theta
    (0, 1),  # R-squared
    (0, 10), # Rho
    (0, 5),  # Sigma
    (0, 1),  # n
    (-10, 10), # x
    (-10, 10), # y
    (0, 50)   # beta
]

# Setting up the axes
for i, ax in enumerate(axs.flat):
    ax.set_title(titles[i])
    ax.set_xlabel('mrVista')
    ax.set_ylabel('Popeye')
    ax.plot(limits[i], limits[i], 'k--')

# Populating the scatter plots
for ix in range(visual_rois.shape[0]):
    for iy in range(visual_rois.shape[1]):
        for iz in range(visual_rois.shape[2]):
            if visual_rois[ix, iy, iz] == 1:
                for i, ax in enumerate(axs.flat):
                    ax.scatter(mrVista_fit[ix, iy, iz, i], RF_ss5_fFit[ix, iy, iz, i], c='r', s=2)

plt.tight_layout()
plt.show()